# Overpass/Overpy learning
Using Overpy to access data from OpenStreetMap.

In [1]:
!pwd

/Users/trevorkinsey/reliance/trevor/mapping


In [62]:
import pandas as pd
import requests
import json
import folium
from folium.features import DivIcon
import overpy
import geopandas as gpd
import contextily as cx

import pickle
import webbrowser

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

api = overpy.Overpass()

In [63]:
# load map tiles   
with open("tiles.json") as f:
    tiles = json.load(f)
tiles = {name: folium.TileLayer(tiles = data["url"],
                                attr = data["attr"], 
                                name = name, 
                                overlay = False, 
                                control = True, 
                                show = True
                               ) for name, data in tiles.items()}
tiles

{'Stamen Toner': <folium.raster_layers.TileLayer at 0x138186970>,
 'Stamen Toner Lite': <folium.raster_layers.TileLayer at 0x139730df0>,
 'Stamen Terrain': <folium.raster_layers.TileLayer at 0x13a53a100>,
 'Stamen Watercolor': <folium.raster_layers.TileLayer at 0x139730fd0>,
 'Google Maps': <folium.raster_layers.TileLayer at 0x13921c730>,
 'Google Satellite': <folium.raster_layers.TileLayer at 0x13921cee0>,
 'Google Satellite Hybrid': <folium.raster_layers.TileLayer at 0x13921c3d0>,
 'Esri Satellite': <folium.raster_layers.TileLayer at 0x13921c910>,
 'Cartodb Positron': <folium.raster_layers.TileLayer at 0x13921ce20>}

In [64]:
# Load nation lakes coords   
with open("nation_lakes.json") as f:
    lakes = json.load(f)
for i,lake in enumerate(lakes):
    print(i,lake["name"])

0 Tsayta Lake
1 Indata Lake
2 Tchentlo Lake
3 Chuchi Lake


In [65]:
lakes[0]

{'name': 'Tsayta Lake',
 'north': 55.48292,
 'west': -125.644,
 'south': 55.41749450097668,
 'east': -125.31534497067527,
 'lat': 55.45020725048834,
 'lon': -125.47967248533763}

In [66]:
df = pd.DataFrame(lakes)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.crs = "WGS84" # web mercator 'EPSG:4326'
gdf

,name,north,west,south,east,lat,lon,geometry
0,Tsayta Lake,55.482920,-125.644000,55.417495,-125.315345,55.450207,-125.479672,POINT (-125.47967 55.45021)
1,Indata Lake,55.392500,-125.320000,55.292000,-125.217500,55.342250,-125.268750,POINT (-125.26875 55.34225)
2,Tchentlo Lake,55.277247,-125.287922,55.164543,-124.759969,55.220895,-125.023945,POINT (-125.02395 55.22089)
3,Chuchi Lake,55.222106,-124.764076,55.148379,-124.310959,55.185243,-124.537517,POINT (-124.53752 55.18524)


In [67]:
def build_query(corners, desired_tags = None):
    query = f"""
nwr{corners}->.all;
(
nwr.all;
);
out center;
    """
    if desired_tags!=None:
        tag_str = ""
        for key, values in desired_tags.items():
            for value in values:
                tag_str += f'nwr.all["{key}"~"{value}"];\n'
        query = query.replace("nwr.all;", tag_str)
    return query


corners = (lake["south"],lake["west"],lake["north"],lake["east"])
desired_tags = {
    'boundary': ['national_park',],
    'leisure': ['nature_reserve'],
    'tourism': ['camp_site'],
    'natural': ['water'],
    'water': ['.'],
}
query = build_query(corners, desired_tags)

In [68]:
print(query)
result = api.query(query)
print(len(result.nodes), "nodes")
print(len(result.ways), "ways")
print(len(result.relations), "relations")


nwr(55.14837905899054, -124.76407610865795, 55.22210608559387, -124.31095886659628)->.all;
(
nwr.all["boundary"~"national_park"];
nwr.all["leisure"~"nature_reserve"];
nwr.all["tourism"~"camp_site"];
nwr.all["natural"~"water"];
nwr.all["water"~"."];

);
out center;
    
12 nodes
97 ways
2 relations


In [69]:
for relation in result.get_relations():
    print(relation.tags)
# result.get_way(184813494, resolve_missing=True)

{'boundary': 'national_park', 'leisure': 'nature_reserve', 'name': 'Nation Lakes Provincial Park', 'source': 'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas', 'type': 'multipolygon', 'wikidata': 'Q6970045', 'wikipedia': 'en:Nation Lakes Provincial Park'}
{'name': 'Nation Lakes', 'natural': 'water', 'source': 'NRCan-CanVec-10.0', 'type': 'multipolygon', 'water': 'lake'}


In [70]:
def get_nwr_in_bbox(corners, buffer = 0, key=None, value=None):
    """Gets all Nodes, Ways, Relations inside bounding box, optionally with specified tag
    PARAMETERS:
        corners(tuple):   limits of bounding box (south,west,north,east)
        tags(dict):  tags to retrieve within bbox (default = {}, returns all Nodes, Ways, Relations)
                     eg {'tourism': ['camp_site'], 'name': ['Ahdatay', 'Pine Point']}
    RETURNS:
        Overpy.Result object"""
    
    buffer = (-buffer, -buffer, buffer, buffer)
    corners = tuple([c+b for c,b in zip(corners, buffer)])
    
    if (key,value) != (None,None):
        query = f"""
        [out:json];
        nwr["{key}"="{value}"] {corners};
        (._;>;);
        out body;"""
    else:
        query = f"""
        [out:json];
        nwr {corners};
        (._;>;);
        out body;"""
    print(query)
    result = api.query(query)
    
    return result


def get_unique_tags(items):
    """Finds every unique tag in list of Nodes, Ways, or Relations
    PARAMETERS:
        items(list):  list of Nodes, Ways, or Relations from Overpass.Result, eg. Overpass.Result.nodes
    RETURNS:
        dict:        eg.{key1: [value1, value2], key2: [value3]}
    """
    tags = {}
    for item in items:
        if item.tags!={}:
            for key,value in item.tags.items():
                if key not in tags:
                    tags[key] = [value]
                elif value not in tags[key]:
                    tags[key].append(value)
    return tags



def select_nwrs_with_tags(nwrs, tags):
    """Selects only the nwrs (Nodes, Ways, or Relations) that contain the desired key:value tags
    PARAMETERS:
        nwrs(list):   list of Nodes, Ways, or Relations
        tags(dict):   tags that we want to find in nwrs eg.{key1: [value1, value2], key2: [value3]}
    RETURNS:
        list:         just the nwrs with desired tags
    """
    # make a list of every key-value pair
    key_value_pairs = []
    for key, values in tags.items():
        for value in values:
            key_value_pairs.append((key,value))

    # select only nwrs with tags in key_value_pairs
    desired_nwrs = []
    for nwr in nwrs:
        if nwr.tags!={}:
            for key, value in nwr.tags.items():
                if ((key,value) in key_value_pairs) & (nwr not in desired_nwrs):
                    desired_nwrs.append(nwr)

    return desired_nwrs    

In [71]:
def make_map(place, tiles, base_tile_name = "Stamen Toner Lite"):
    #Create the map
    map_centre = [place["lat"], place["lon"]]
    zoom_level = cx.tile._calculate_zoom(place["west"], place["south"], place["east"], place["north"])
    m = folium.Map(
        location = map_centre, 
        zoom_start = zoom_level-2, 
        min_zoom = 8, 
        max_zoom = 22, 
        zoomSnap = 0.25,
        zoomDelta = 0.25,
        tiles=None,
        control_scale=True,
        zoom_control=False,
        boxZoom = True,
    )
    # add tiles
    for tile_name in tiles:
        tile_layer = tiles[tile_name]
        if tile_name == base_tile_name:
            tile_layer.overlay = False  
            tile_layer.show = True      # shows on load
            tile_layer.add_to(m)
            # tile_layer.opacity = 0.5
        else:
            tile_layer.overlay = True
            tile_layer.show = False
        tile_layer.add_to(m)
    # add layer control element
    folium.LayerControl().add_to(m)
    
    # m.fit_bounds([place["west"], place["south"]], [place["east"], place["north"]])
    
    return m

def add_node(m, node, hover_text = None, **style):
    if ("natural", "peak") in node.tags.items():
        name = node.tags["name"] if "name" in node.tags else ""
        if "Range" in name:
            marker = None
        else:
            icon=DivIcon(icon_size=(200,10), icon_anchor=(0,0), 
                         html=f'<div style="font-size: {style["font_size"]}pt; color:{style["color"]}">+ {name}</div>',)
            marker = folium.map.Marker([node.lat, node.lon], 
                                       icon=icon, 
                                       tooltip =  hover_text if hover_text!=None else None,)
    else:
        marker = folium.CircleMarker(location=[node.lat, node.lon], 
                        **style, 
                        tooltip =  hover_text if hover_text!=None else None,
                        )
    if marker != None:
        marker.add_to(m) 
    return m


def add_way(m, way, hover_text = None, **style):
    folium.PolyLine([(float(node.lat), float(node.lon)) for node in way.nodes], # list of (lat,lon) tuples
                    **style,
                    opacity=0.8,
                    tooltip =  hover_text if hover_text!=None else None,
                    ).add_to(m)
    return m


def add_relation(m, relation, hover_text = None):
    style = set_marker_style(relation)
    hover_text = write_hover_text(relation)
    for member in relation.members:
        
        if type(member)==overpy.RelationNode:
            node = result.get_node(member.ref, resolve_missing = True)
            m = add_node(m, node, hover_text = hover_text, **style)
        elif type(member)==overpy.RelationWay:
            way = result.get_way(member.ref, resolve_missing = True)
            m = add_way(m, way, hover_text = hover_text, **style)
    return m


def write_hover_text(nwr):
    tags = nwr.tags
    hover_items = []
    hover_text = str(nwr.id)+" :"
    if "name" in tags:
        hover_items.append(tags["name"])
    if ("tourism", "camp_site") in tags.items():
        hover_items.append("Campsite")
    
    if len(hover_items)>0:
        hover_text += "</br>".join(hover_items)
    # else: 
    hover_text += json.dumps(tags)
    return hover_text


def set_marker_style(nwr,default_color = "maroon", default_weight = 1, default_radius = 7):
    style = {
        "color": default_color,
        "weight": default_weight,
    }
    if type(nwr) == overpy.Node:
        style["radius"] = default_radius
    tags = nwr.tags
    items = tags.items()
    for item in [('natural','water'),('natural','coastline')]:
        if item in items:
            style["color"] = "blue"
            if type(nwr) in [overpy.Way, overpy.Relation]:
                style["fill_color"]='blue'
                style["fill_opacity"]=0
    for key in ['water', 'waterway']:
        if key in tags:
            style["color"] = "blue"
            
    if ("tourism","camp_site") in items:
        style["color"]="green"
        style["weight"] = 1.5
    
    if 'highway' in tags:
        style["color"] = "grey"
    for item in [('highway', 'path'), ('highway', 'footway')]:
        if item in items:
            style["dash_array"]='3'
    for item in [('leisure', 'nature_reserve'), ('leisure', 'park')]:
        if item in items:
            style["color"]='green'
            style["fill_color"]='green'
            style["fill_opacity"]=0.1
            
    if ("natural", "peak") in items:
        style["color"]='black'
        style["font_size"]=8

    return style

In [72]:
# Make a lake that encompasses all the lakes
lake = {
    'name': 'Nation Lakes',
    'north': max([l["north"] for l in lakes]),
    'west': min([l["west"] for l in lakes]),
    'south': min([l["south"] for l in lakes]),
    'east': max([l["east"] for l in lakes]),
}
lake["lat"] = (lake["north"] + lake["south"])/2
lake["lon"] = (lake["east"] + lake["west"])/2
lake

{'name': 'Nation Lakes',
 'north': 55.48292,
 'west': -125.644,
 'south': 55.14837905899054,
 'east': -124.31095886659628,
 'lat': 55.315649529495275,
 'lon': -124.97747943329814}

In [73]:
print(lake["name"])
corners = (lake["south"],lake["west"],lake["north"],lake["east"])

result_file = "nation-lake-result.pkl"

# # get ALL nodes, ways, relations
# result = get_nwr_in_bbox(corners, buffer = 0.02)

# # save result to file
# with open(result_file, "wb") as f:
#     pickle.dump(result, f)
# print(f"Overpass result saved to: {result_file}")

# load result from file
with open(result_file, "rb") as f:
    result = pickle.load(f)

print(len(result.nodes), "nodes")
print(len(result.ways), "ways")
print(len(result.relations), "relations")

Nation Lakes
432529 nodes
11867 ways
101 relations


## Nodes

In [74]:
# nodes = get_unique_tags(result.nodes)
# for key in ["name",'name:fr','name:en','source','ref','alt_name','start_date', 'ele','format']:
#     del nodes[key]
# nodes

In [75]:
desired_tags = {
    'natural': ['spring', 'peak'],
    'tourism': ['camp_site'],
    'barrier': ['yes'],
}
nodes = select_nwrs_with_tags(result.nodes, desired_tags)
print(len(nodes), "nodes found with those tags")

182 nodes found with those tags


In [76]:
nodes

[<overpy.Node id=244245903 lat=56.2650369 lon=-130.6154490>,
 <overpy.Node id=244245911 lat=56.3668442 lon=-130.7819800>,
 <overpy.Node id=244245929 lat=56.4060047 lon=-131.0873480>,
 <overpy.Node id=244245944 lat=56.4495239 lon=-131.1734660>,
 <overpy.Node id=244245984 lat=56.6122319 lon=-131.5810670>,
 <overpy.Node id=244245987 lat=56.5525902 lon=-131.4718290>,
 <overpy.Node id=244245996 lat=56.5992028 lon=-131.8353270>,
 <overpy.Node id=244246014 lat=56.7029842 lon=-131.8606690>,
 <overpy.Node id=244246039 lat=56.7536750 lon=-131.9010717>,
 <overpy.Node id=244246043 lat=56.8736483 lon=-132.1235670>,
 <overpy.Node id=244246063 lat=57.0450592 lon=-132.0451910>,
 <overpy.Node id=244246084 lat=57.0915128 lon=-132.3689831>,
 <overpy.Node id=244246106 lat=57.2112533 lon=-132.2479210>,
 <overpy.Node id=244246111 lat=57.3497467 lon=-132.3697910>,
 <overpy.Node id=244246162 lat=57.4965136 lon=-132.5536840>,
 <overpy.Node id=244246193 lat=57.6167369 lon=-132.6608640>,
 <overpy.Node id=2442462

## Ways

In [77]:
ways = get_unique_tags(result.ways)
# print(ways["name"])
# for key in ["name",'source']:
#     del ways[key]
# ways

In [78]:
desired_tags = {
    'name': ['Tsayta Lake', 'Indata Lake', 'Tchentlo Lake', 'Chuchi Lake', 'Nation Lakes Provincial Park', 'Nation River'],
    'leisure': ['park'],
    'natural': ['peak', 'saddle', 'mountain_range', 'valley', 'volcano', 'spring', 'land'],
    'highway': ['path', 'footway', 'unclassified', 'tertiary'],
    'surface': ['unpaved'],
}

ways = select_nwrs_with_tags(result.ways, desired_tags)
print(len(ways), "ways found with those tags")

219 ways found with those tags


In [79]:
way = ways[0]
way

<overpy.Way id=168317108 nodes=[1796040054, 1796040034, 1796040035, 1796040038, 1796040040, 1796040044, 1796040045, 1796040048, 1796040050, 1796040049, 1796040052, 1796040051, 1796040046, 1796040042, 1796040039, 1796040036, 1796040033, 1796040031, 1796040030, 1796040032, 1796040037, 1796040041, 1796040043, 1796040047, 1796040053, 1796040055, 1796040056, 1796040057, 1796040054]>

In [80]:
result.get_node(1796040054)


<overpy.Node id=1796040054 lat=55.2029906 lon=-124.7945780>

In [81]:
way.get_node(1796040054)

AttributeError: 'Way' object has no attribute 'get_node'

## Relations

In [82]:
# get_unique_tags(result.relations)

In [83]:
desired_tags = {
    'name': ['Tsayta Lake', 'Indata Lake', 'Tchentlo Lake', 'Chuchi Lake', 'Nation Lakes Provincial Park', 'Nation Lakes'],
    'leisure': ['nature_reserve'],
    # 'route': ['hiking'],
    # 'natural': ['water'],
    # 'water': ['river']
}

relations = select_nwrs_with_tags(result.get_relations(), desired_tags)
print(len(relations), "relations found with those tags")
[relation.tags for relation in relations]

6 relations found with those tags


[{'boundary': 'national_park',
  'leisure': 'nature_reserve',
  'name': 'Nation Lakes Provincial Park',
  'source': 'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas',
  'type': 'multipolygon',
  'wikidata': 'Q6970045',
  'wikipedia': 'en:Nation Lakes Provincial Park'},
 {'boundary': 'national_park',
  'leisure': 'nature_reserve',
  'name': 'Takla Marine Provincial Park - Takla West',
  'source': 'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas',
  'type': 'multipolygon'},
 {'name': 'Nation Lakes',
  'natural': 'water',
  'source': 'NRCan-CanVec-10.0',
  'type': 'multipolygon',
  'water': 'lake'},
 {'name': 'Tsayta Lake',
  'natural': 'water',
  'type': 'multipolygon',
  'water': 'lake'},
 {'name': 'Tchentlo Lake',
  'natural': 'water',
  'source': 'NRCan-CanVec-10.0;MAXAR 2018',
  'type': 'multipolygon',
  'water': 'lake'},
 {'name': 'Nation Lakes',
  'natural': 'water',
  'source': 'NRCan-CanVec-10.0',
  'type': 'multipolygon'}]

In [84]:
for i,relation in enumerate(relations):
    print(i, relation.id, relation.tags)

0 2226155 {'boundary': 'national_park', 'leisure': 'nature_reserve', 'name': 'Nation Lakes Provincial Park', 'source': 'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas', 'type': 'multipolygon', 'wikidata': 'Q6970045', 'wikipedia': 'en:Nation Lakes Provincial Park'}
1 2226160 {'boundary': 'national_park', 'leisure': 'nature_reserve', 'name': 'Takla Marine Provincial Park - Takla West', 'source': 'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas', 'type': 'multipolygon'}
2 2463005 {'name': 'Nation Lakes', 'natural': 'water', 'source': 'NRCan-CanVec-10.0', 'type': 'multipolygon', 'water': 'lake'}
3 7403180 {'name': 'Tsayta Lake', 'natural': 'water', 'type': 'multipolygon', 'water': 'lake'}
4 10348632 {'name': 'Tchentlo Lake', 'natural': 'water', 'source': 'NRCan-CanVec-10.0;MAXAR 2018', 'type': 'multipolygon', 'water': 'lake'}
5 15936519 {'name': 'Nation Lakes', 'natural': 'water', 'source': 'NRCan-CanVec-10.0', 'type': 'multipolygon'}


In [85]:
corrections = {
    2463005: {"name": "Chuchi Lake"},
    15936519: {"name": "Indata Lake"}
}
for relation in relations:
    if relation.id in corrections:
        for key, value in corrections[relation.id].items():
            relation.tags[key]=value
            print(relation.id,key, value)

2463005 name Chuchi Lake
15936519 name Indata Lake


In [86]:
relation.members

[<overpy.RelationWay ref=1178486144 role=inner>,
 <overpy.RelationWay ref=1178484294 role=inner>,
 <overpy.RelationWay ref=750576311 role=outer>]

In [87]:
relation.id

15936519

In [88]:
result.get_way(1178484294)

<overpy.Way id=1178484294 nodes=[10947991819, 10948079032, 10948057599, 10948078914, 10948074546, 10948056510, 10948057823, 10947988872, 10948036193, 10948009045, 10948010861, 10948048355, 10948060314, 10948021917, 10948010581, 10948075056, 10948057708, 10948004852, 10948040675, 10948008538, 10948018670, 10948042601, 10947991819]>

## Lake labels

In [89]:
lake_label_info = {}
for l in lakes:
    lake_label_info[l["name"]] = {
        "center": [l["lat"], l["lon"]],
        "offset": (0,0),
        "angle": 0
    }
lake_label_info

{'Tsayta Lake': {'center': [55.45020725048834, -125.47967248533763],
  'offset': (0, 0),
  'angle': 0},
 'Indata Lake': {'center': [55.34225, -125.26875],
  'offset': (0, 0),
  'angle': 0},
 'Tchentlo Lake': {'center': [55.220894912282674, -125.02394547369838],
  'offset': (0, 0),
  'angle': 0},
 'Chuchi Lake': {'center': [55.18524257229221, -124.53751748762711],
  'offset': (0, 0),
  'angle': 0}}

In [90]:
lake_label_info

{'Tsayta Lake': {'center': [55.45020725048834, -125.47967248533763],
  'offset': (0, 0),
  'angle': 0},
 'Indata Lake': {'center': [55.34225, -125.26875],
  'offset': (0, 0),
  'angle': 0},
 'Tchentlo Lake': {'center': [55.220894912282674, -125.02394547369838],
  'offset': (0, 0),
  'angle': 0},
 'Chuchi Lake': {'center': [55.18524257229221, -124.53751748762711],
  'offset': (0, 0),
  'angle': 0}}

In [91]:
lake

{'name': 'Nation Lakes',
 'north': 55.48292,
 'west': -125.644,
 'south': 55.14837905899054,
 'east': -124.31095886659628,
 'lat': 55.315649529495275,
 'lon': -124.97747943329814}

# Make a map wih the NWRs

In [92]:
print(f"tiles: {list(tiles.keys())}")

tiles: ['Stamen Toner', 'Stamen Toner Lite', 'Stamen Terrain', 'Stamen Watercolor', 'Google Maps', 'Google Satellite', 'Google Satellite Hybrid', 'Esri Satellite', 'Cartodb Positron']


In [93]:
def add_nwr(m, nodes, ways, relations):
    for node in nodes:
        node = result.get_node(node.id,resolve_missing=True)       
        hover_text = write_hover_text(node)
        # color, weight = select_color_weight(node.tags)
        style = set_marker_style(node)
        m = add_node(m, node, hover_text, **style)
    for way in ways:
        way = result.get_way(way.id,resolve_missing=True)                       
        hover_text = write_hover_text(way)
        style = set_marker_style(way)
        m = add_way(m, way, hover_text, **style)
    for relation in relations:
        relation = result.get_relation(relation.id,resolve_missing=True)
        # print(relation.id, relation.tags)
        hover_text = write_hover_text(relation)
        m = add_relation(m, relation)

    # m.add_child(folium.LatLngPopup())

    # Map title
    font_color = "black"
    icon=DivIcon(icon_size=(400,30), icon_anchor=(100,250), html=f'<div style="font-size: 50pt; color:{font_color}">{lake["name"]}</div>',)
    folium.map.Marker([lake["lat"], lake["lon"]], icon=icon).add_to(m)
    
    return m





In [94]:
lake_label_info

{'Tsayta Lake': {'center': [55.45020725048834, -125.47967248533763],
  'offset': (0, 0),
  'angle': 0},
 'Indata Lake': {'center': [55.34225, -125.26875],
  'offset': (0, 0),
  'angle': 0},
 'Tchentlo Lake': {'center': [55.220894912282674, -125.02394547369838],
  'offset': (0, 0),
  'angle': 0},
 'Chuchi Lake': {'center': [55.18524257229221, -124.53751748762711],
  'offset': (0, 0),
  'angle': 0}}

In [95]:
def add_labels(m, label_info, font_color = "red", font_size = 30):
    for label,info in label_info.items():
        
        name = label
        icon=DivIcon(icon_size=(400,30), icon_anchor=(0,0), html=f'<div style="font-size: {font_size}pt; color:{font_color}">{name}</div>',)
        
        marker = folium.map.Marker(info["center"], icon=icon, angle = info["angle"])
        marker.add_to(m)
        print(label,marker)
    return m

In [96]:
base_tile_name = 'Stamen Toner Lite'
m = make_map(lake, tiles, base_tile_name)
m = add_nwr(m, nodes, ways, relations)
m = add_labels(m, lake_label_info, font_size = 30)

m

Tsayta Lake <folium.map.Marker object at 0x13a6bd9a0>
Indata Lake <folium.map.Marker object at 0x13a6c83a0>
Tchentlo Lake <folium.map.Marker object at 0x13a6c8550>
Chuchi Lake <folium.map.Marker object at 0x13a7ac8e0>


In [ ]:
# save to html
outfile= "NationLakes.html"
m.save(outfile= outfile)

In [41]:
m.width

(100.0, '%')

In [66]:
# save to html
m.save(outfile= "NationLakes.html")

In [325]:
# Save to png

import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(f'{lake["name"]}.png')

In [116]:
result = get_nwr_in_bbox(lake["south"],lake["west"],lake["north"],lake["east"], key="tourism", value="camp_site")


In [7]:
nodes

[<overpy.Node id=7579029096 lat=55.1844698 lon=-125.1106256>,
 <overpy.Node id=7579029097 lat=55.2018961 lon=-124.8032820>,
 <overpy.Node id=7579029098 lat=55.2224236 lon=-124.8436385>,
 <overpy.Node id=7579029099 lat=55.2105895 lon=-124.8862749>,
 <overpy.Node id=7579029100 lat=55.2265021 lon=-124.8984575>,
 <overpy.Node id=7579029101 lat=55.2206274 lon=-124.9142933>,
 <overpy.Node id=7579029102 lat=55.2135675 lon=-124.9466729>,
 <overpy.Node id=7579029103 lat=55.1913451 lon=-124.9725616>,
 <overpy.Node id=7579029105 lat=55.1962872 lon=-125.0631934>,
 <overpy.Node id=7579029106 lat=55.1838174 lon=-125.1274645>,
 <overpy.Node id=7579029107 lat=55.1961402 lon=-125.1348352>,
 <overpy.Node id=7579029108 lat=55.1816627 lon=-125.1413396>,
 <overpy.Node id=7579029109 lat=55.2023247 lon=-125.1553703>,
 <overpy.Node id=7579029110 lat=55.2004204 lon=-125.1911402>,
 <overpy.Node id=7579029111 lat=55.2193307 lon=-125.2340998>,
 <overpy.Node id=7579029114 lat=55.2294238 lon=-125.2479279>,
 <overpy

In [8]:
result.ways



[]

In [109]:
result.relations


[<overpy.Relation id=8515761>]

In [107]:
type(result)

overpy.Result

In [107]:
# Save to png

# import io
# from PIL import Image

# img_data = m._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save('folium-export.png')